In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

**data import**

In [2]:
evi =  pd.read_csv('evi_summaryeq0.csv')
ndvi =  pd.read_csv('ndvi_summaryeq0.csv')

In [3]:
#행정동코드
ref_Hcode = pd.read_excel('./DB 시군구 코드 기준_1995_2019.xlsx')
col = list(map(str, ref_Hcode.columns))
col = [c.strip() for c in col]

ref_Hcode.columns = col
ref_Hcode = ref_Hcode.rename(columns={'시도 코드':'sido_CD'})

ref_Hcode.head(5)

,시도,sido_CD,시군구,기준 코드\n(=2019),2019,2018,2017,2016,2015,2014,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,비고
0,서울특별시,11,종로구,11010,11010,11010,11010,11010,11010,11010,...,11010.0,11010.0,11010,11010,11010,11010,11010,11010,11010,NaN
1,서울특별시,11,중구,11020,11020,11020,11020,11020,11020,11020,...,11020.0,11020.0,11020,11020,11020,11020,11020,11020,11020,NaN
2,서울특별시,11,용산구,11030,11030,11030,11030,11030,11030,11030,...,11030.0,11030.0,11030,11030,11030,11030,11030,11030,11030,NaN
3,서울특별시,11,성동구,11040,11040,11040,11040,11040,11040,11040,...,11040.0,11040.0,11040,11040,11040,11040,11040,11040,11040,NaN
4,서울특별시,11,광진구,11050,11050,11050,11050,11050,11050,11050,...,11050.0,11050.0,11050,11050,11050,11050,11050,11050,11050,NaN


In [4]:
ref_Hcode = ref_Hcode[['시도','시군구','2019']]
ref_Hcode['NM'] = ref_Hcode['시도'].str.strip() + ' ' + ref_Hcode['시군구'].str.strip()
ref_Hcode.head()

,시도,시군구,2019,NM
0,서울특별시,종로구,11010,서울특별시 종로구
1,서울특별시,중구,11020,서울특별시 중구
2,서울특별시,용산구,11030,서울특별시 용산구
3,서울특별시,성동구,11040,서울특별시 성동구
4,서울특별시,광진구,11050,서울특별시 광진구


**특정 지역 매칭코드**

In [5]:
# 하나씩 탐색
ref_Hcode.loc[ref_Hcode['NM'].str.startswith("경기도 안양시")]

,시도,시군구,2019,NM
83,경기도,안양시 만안구,31041,경기도 안양시 만안구
84,경기도,안양시 동안구,31042,경기도 안양시 동안구


**NAME 수정**

In [6]:
evi.head()

,year,SIGUNGU_CD,SIDO_NM,SIGUNGU_NM,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,2000,11010,서울,종로구,NaN,660.912206,972.406495,1267.180055,2565.266805,2775.509520,3931.680626,2792.363850,2537.233568,1744.897542,1226.778951,1097.512846
1,2000,11020,서울,중구,NaN,597.457935,523.735837,926.007543,1315.420290,1549.399169,NaN,1567.668415,1395.397783,987.166357,684.938424,587.369150
2,2000,11030,서울,용산구,NaN,708.946770,549.234091,830.418915,1026.549107,1354.420748,NaN,1270.221265,1438.377713,1082.770601,758.087243,676.220381
3,2000,11040,서울,성동구,NaN,501.189474,423.850910,630.582193,1052.754356,1196.280540,NaN,1316.687220,1250.784106,934.674446,617.065491,476.676863
4,2000,11050,서울,광진구,NaN,591.941241,576.194522,660.178642,1701.907855,1459.960147,NaN,1362.892907,1464.783445,1110.814168,759.291082,589.183462


In [7]:
evi.SIDO_NM.unique()

array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남',
       '전북', '전남', '경북', '경남', '제주'], dtype=object)

In [8]:
ref_Hcode.시도.unique()


array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [9]:
replace_list = {'서울':'서울특별시', '부산':'부산광역시', '대구':'대구광역시', '인천':'인천광역시', '광주':'광주광역시', '대전':'대전광역시', '울산':'울산광역시',
                '경기':'경기도', '강원':'강원도', '충북':'충청북도', '충남':'충청남도', '전북':'전라북도', '전남':'전라남도', '경북':'경상북도',
                '경남':'경상남도','제주':'제주특별자치도'}

evi['SIDO_NM'] = evi['SIDO_NM'].replace(replace_list)
ndvi['SIDO_NM'] = ndvi['SIDO_NM'].replace(replace_list)

EVI

In [10]:
# name에 띄어쓰기가 없음 -> 띄어쓰기 포함해주기
naming = evi['SIGUNGU_NM'].str.partition('시')
naming.loc[naming[1] == '시', 'name'] = naming[0] + naming[1] + " " + naming[2]
naming.loc[naming[1] != '시', 'name'] = naming[0]

naming.tail()

,0,1,2,name
5747,함양군,,,함양군
5748,거창군,,,거창군
5749,합천군,,,합천군
5750,제주,시,,제주시
5751,서귀포,시,,서귀포시


In [11]:
evi['SIGUNGU_NM_new'] = naming.name
evi[['SIGUNGU_NM',"SIGUNGU_NM_new"]].loc[evi['SIGUNGU_NM_new'].str.startswith('수원시')].head()

,SIGUNGU_NM,SIGUNGU_NM_new
73,수원시장안구,수원시 장안구
74,수원시권선구,수원시 권선구
75,수원시팔달구,수원시 팔달구
76,수원시영통구,수원시 영통구
324,수원시장안구,수원시 장안구


NDVI

In [12]:
naming = ndvi['SIGUNGU_NM'].str.partition('시')
naming.loc[naming[1] == '시', 'name'] = naming[0] + naming[1] + " " + naming[2]
naming.loc[naming[1] != '시', 'name'] = naming[0]

ndvi['SIGUNGU_NM_new'] = naming.name
ndvi[['SIGUNGU_NM',"SIGUNGU_NM_new"]].loc[ndvi['SIGUNGU_NM_new'].str.startswith('수원시')].head()

,SIGUNGU_NM,SIGUNGU_NM_new
73,수원시장안구,수원시 장안구
74,수원시권선구,수원시 권선구
75,수원시팔달구,수원시 팔달구
76,수원시영통구,수원시 영통구
324,수원시장안구,수원시 장안구


In [13]:
evi['NAME'] = evi['SIDO_NM'] + " " + evi['SIGUNGU_NM_new']
ndvi['NAME'] = ndvi['SIDO_NM'] + " " + ndvi['SIGUNGU_NM_new']

evi['NAME'] = evi['NAME'].str.strip()
ndvi['NAME'] = ndvi['NAME'].str.strip()

### **EVI+NDVI**

In [14]:
evi = evi.sort_values(by=['year','SIGUNGU_CD'])
ndvi = ndvi.sort_values(by=['year','SIGUNGU_CD'])

In [15]:
# Reshape and rename the EVI columns 
evi_re = evi[['SIGUNGU_CD','NAME']]

clist = [c for c in evi.columns if 'month' in c]

for y in range(2015, 2023):
    new_clist = [f'{y}_{c}_EVI' for c in evi.columns if 'month' in c]
    
    sub = evi.loc[evi.year==y].drop(columns=['year','SIDO_NM','SIGUNGU_NM','SIGUNGU_NM_new','NAME']).reset_index(drop=True)
    sub[clist] = sub[clist]/10000 # 단위 변경
    sub.columns = ['SIGUNGU_CD'] + new_clist
    
    evi_re = evi_re.merge(sub, on='SIGUNGU_CD', how='left')

In [16]:
evi_renew = evi_re.rename(columns={'SIGUNGU_CD':'SIGUNGU_CD_EVI','NAME':'NAME_EVI'})
evi_renew =  evi_renew.drop_duplicates(subset='SIGUNGU_CD_EVI')

In [ ]:
evi_renew[evi_renew['NAME_EVI'].str.startswith('제주')]

In [18]:
# Reshape and rename the NDVI columns 

ndvi_re = ndvi[['SIGUNGU_CD','NAME']]
clist = [c for c in ndvi.columns if 'month' in c]

for y in range(2015, 2023):
    new_clist = [f'{y}_{c}_NDVI' for c in ndvi.columns if 'month' in c]
    
    sub = ndvi.loc[ndvi.year==y].drop(columns=['year','SIDO_NM','SIGUNGU_NM','SIGUNGU_NM_new','NAME']).reset_index(drop=True)
    sub[clist] = sub[clist]/10000 # 단위 변경
    sub.columns = ['SIGUNGU_CD'] + new_clist
    
    ndvi_re = ndvi_re.merge(sub, on='SIGUNGU_CD', how='left')

In [20]:
ndvi_renew = ndvi_re.rename(columns={'SIGUNGU_CD':'SIGUNGU_CD_NDVI','NAME':'NAME_NDVI'})
ndvi_renew =  ndvi_renew.drop_duplicates(subset='SIGUNGU_CD_NDVI')

In [23]:
evi_renew = evi_renew.sort_values(by='SIGUNGU_CD_EVI')
ndvi_renew = ndvi_renew.sort_values(by='SIGUNGU_CD_NDVI')

veg = pd.concat([evi_renew,ndvi_renew], axis=1)
veg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 249
Columns: 196 entries, SIGUNGU_CD_EVI to 2022_month12_NDVI
dtypes: float64(192), int64(2), object(2)
memory usage: 386.3+ KB


In [24]:
veg.to_excel('EVI_and_NDVI.xlsx', index=False, header=True, encoding='euc-kr')

### **행정동코드 merge**

In [25]:
vegetation = pd.read_excel('EVI_and_NDVI.xlsx')

vegetation = vegetation.drop(columns=['NAME_NDVI']).rename(columns={'NAME_EVI':'NAME_VEG', 'SIGUNGU_CD_EVI':'Hcode_VEG'})

**특정 지역명 수정 및 생성:**

In [26]:
vegetation[vegetation['NAME_VEG'].str.startswith('충청북도 청주시')]['NAME_VEG']

136    충청북도 청주시 상당구
137    충청북도 청주시 흥덕구
Name: NAME_VEG, dtype: object

충청북도 청주시 생성

In [ ]:
cheongju = pd.DataFrame(vegetation.loc[vegetation['NAME_VEG'].str.startswith('충청북도 청주시'),
                             [c for c in vegetation.columns if 'month' in c]].mean(axis=0)).transpose()

cheongju['NAME_VEG'] = '충청북도 청주시'
cheongju['Hcode_VEG'] = 33040

vegetation = vegetation.append(cheongju).reset_index(drop=True)

vegetation.tail()

충청북도 청주시 서원구 생성(서원구=흥덕구)

In [ ]:
cheongju_seowongu = pd.DataFrame(vegetation.loc[vegetation['NAME_VEG'].str.contains('충청북도 청주시 흥덕구')])

cheongju_seowongu['NAME_VEG'] = '충청북도 청주시 서원구'
cheongju_seowongu['Hcode_VEG'] = 33042

vegetation = vegetation.append(cheongju_seowongu).reset_index(drop=True)

vegetation.tail()


**행정코드와 연계**

In [29]:
ref_Hcode.head()

,시도,시군구,2019,NM
0,서울특별시,종로구,11010,서울특별시 종로구
1,서울특별시,중구,11020,서울특별시 중구
2,서울특별시,용산구,11030,서울특별시 용산구
3,서울특별시,성동구,11040,서울특별시 성동구
4,서울특별시,광진구,11050,서울특별시 광진구


In [30]:
code = ref_Hcode[['NM','2019']].astype(str).rename(columns= {'NM':'행정동이름', '2019':'행정동코드'})

code.head()

,행정동이름,행정동코드
0,서울특별시 종로구,11010
1,서울특별시 중구,11020
2,서울특별시 용산구,11030
3,서울특별시 성동구,11040
4,서울특별시 광진구,11050


In [31]:
# 시흥시만 수정
vegetation.loc[vegetation['NAME_VEG'] == '경기도 시 흥시', "NAME_VEG"] = '경기도 시흥시'

vegetation['NAME_VEG'] = vegetation['NAME_VEG'].str.strip()
code['행정동이름'] = code['행정동이름'].str.strip()

In [32]:
mapping_veg = vegetation.merge(code[['행정동이름','행정동코드']], left_on='NAME_VEG', right_on='행정동이름', how='left')

In [33]:
mapping_veg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253 entries, 0 to 252
Columns: 197 entries, Hcode_VEG to 행정동코드
dtypes: float64(193), int64(1), object(3)
memory usage: 391.4+ KB


In [34]:
# 맵핑되지 못한 지역명 확인: 
mapping_veg.loc[mapping_veg['행정동이름'].isnull()]['NAME_VEG'].unique()
# 경기도 부천시 소사구~원미구 -> 경기도 부천시 로 통일
# 경기도 여주군 -> 경기도 여주시 로 변경
# 인천광역시 남구 -> 인천광역시 미추홀구 로 변경
# 충청남도 당진군 -> 충청남도 당진시 로 변경
# 충청남도 연기군 -> 충청남도 연기시 -> 세종특별자치시 세종시 로 변경
# 충청북도 청원군 -> 충청북도 청원구 로 변경

array(['인천광역시 남구', '경기도 부천시 원미구', '경기도 부천시 소사구', '경기도 부천시 오정구', '경기도 여주군',
       '충청북도 청원군', '충청남도 연기군', '충청남도 당진군'], dtype=object)

**특정 지역명 수정**

In [35]:
# 경기도 부천시 소사구~원미구 -> 경기도 부천시
# 경기도 여주군 -> 경기도 여주시 
# 인천광역시 남구 -> 인천광역시 미추홀구
# 충청남도 당진군 -> 충청남도 당진시
# 충청남도 연기군 -> 충청남도 연기시 -> 세종특별자치시 세종시
# 충청북도 청원군 -> 충청북도 청원구

vegetation['newNAME_VEG'] = vegetation['NAME_VEG']
vegetation['newNAME_VEG'] = vegetation['newNAME_VEG'].replace({'경기도 부천시 소사구':'경기도 부천시', '경기도 부천시 원미구':'경기도 부천시', '경기도 부천시 오정구':'경기도 부천시',
                                                               
                                                               '경기도 여주군':'경기도 여주시', '충청남도 당진군':'충청남도 당진시', '인천광역시 남구':'인천광역시 미추홀구', 
                                                               
                                                               '충청남도 연기군':'세종특별자치시 세종시', '충청북도 청원군':'충청북도 청주시 청원구'})

In [36]:
mapping_veg_new = vegetation.merge(code[['행정동이름','행정동코드']], left_on='newNAME_VEG', right_on='행정동이름', how='left')

pd.set_option('display.max_rows', 30)
mapping_veg_new.isnull().sum()

Hcode_VEG              0
NAME_VEG               0
2015_month1_EVI        4
2015_month2_EVI        3
2015_month3_EVI        7
                    ... 
2022_month11_NDVI    253
2022_month12_NDVI    253
newNAME_VEG            0
행정동이름                  0
행정동코드                  0
Length: 198, dtype: int64

In [37]:
mapping_veg_new.to_excel('EVI,NDVI_월별_행정동연계.xlsx', encoding='euc-kr')

### **법정동코드 merge**

In [38]:
pd.set_option('display.max_rows', 30)

In [39]:
#법정동코드
ref_Bcode = pd.read_excel('./jscode20210601/KIKcd_H.20210601.xlsx')
ref_Bcode = ref_Bcode.loc[ref_Bcode['읍면동명'].isnull(), :].reset_index(drop=True)

ref_Bcode.loc[ref_Bcode['시군구명'].notnull(), '법정동이름'] = ref_Bcode['시도명'].str.strip() + ' ' + ref_Bcode['시군구명'].str.strip()
ref_Bcode.loc[ref_Bcode['시군구명'].isnull(), '법정동이름'] = ref_Bcode['시도명'].str.strip()

ref_Bcode['법정동코드'] = ref_Bcode['행정동코드'].astype(str).apply(lambda x: x[:5])
ref_Bcode.head()

,행정동코드,시도명,시군구명,읍면동명,생성일자,말소일자,법정동이름,법정동코드
0,1100000000,서울특별시,NaN,NaN,19880423,NaN,서울특별시,11000
1,1111000000,서울특별시,종로구,NaN,19880423,NaN,서울특별시 종로구,11110
2,1114000000,서울특별시,중구,NaN,19880423,NaN,서울특별시 중구,11140
3,1117000000,서울특별시,용산구,NaN,19880423,NaN,서울특별시 용산구,11170
4,1120000000,서울특별시,성동구,NaN,19880423,NaN,서울특별시 성동구,11200


In [40]:
# 하나씩 탐색
ref_Bcode.loc[ref_Bcode['법정동이름'].str.startswith("경기도")].head()

,행정동코드,시도명,시군구명,읍면동명,생성일자,말소일자,법정동이름,법정동코드
86,4100000000,경기도,NaN,NaN,19880423,NaN,경기도,41000
88,4111000000,경기도,수원시,NaN,19880423,NaN,경기도 수원시,41110
89,4111100000,경기도,수원시 장안구,NaN,19880701,NaN,경기도 수원시 장안구,41111
90,4111300000,경기도,수원시 권선구,NaN,19880701,NaN,경기도 수원시 권선구,41113
91,4111500000,경기도,수원시 팔달구,NaN,19930115,NaN,경기도 수원시 팔달구,41115


In [41]:
# 세종특별자치시 2개 존재 > 하나 지움
ref_Bcode = ref_Bcode.drop_duplicates(subset='법정동코드', keep='first')

In [42]:
# 세종특별자치시 세종시(법정코드) -> 세종특별자치시(행정코드) 연계 위해서 수정
mapping_veg_new['newNAME_VEG'] = mapping_veg_new['newNAME_VEG'].replace({'세종특별자치시 세종시':'세종특별자치시'})

In [43]:
total_veg = mapping_veg_new.drop(columns=['NAME_VEG','행정동이름']).merge(ref_Bcode[['법정동이름','법정동코드']], left_on='newNAME_VEG', right_on='법정동이름', how='left')

total_veg.isnull().sum()

Hcode_VEG              0
2015_month1_EVI        4
2015_month2_EVI        3
2015_month3_EVI        7
2015_month4_EVI        3
                    ... 
2022_month12_NDVI    254
newNAME_VEG            0
행정동코드                  0
법정동이름                  0
법정동코드                  0
Length: 198, dtype: int64

In [47]:
total_veg = total_veg.rename(columns={'법정동이름':'법정동=행정동이름'})

total_veg.loc[total_veg['법정동=행정동이름'] == '충청북도 청주시', '행정동코드'] = 33040

In [49]:
total_veg.to_excel('Fin_EVI,NDVI_시군구별(1-12월).xlsx',index=False, header=True)